# Traditional ML Models for ZKML: XGBoost

*In this series of tutorials, we delve into the world of traditional machine learning models for ZKML. Despite the hype surrounding advanced AI techniques, traditional ML models often offer superior performance or sufficiently robust results for specific applications. This is particularly true for ZKML use cases, where computational proof costs can be a critical factor. We aim to equip you with guides on how to implement machine learning algorithms suitable for Giza platform applications. This includes practical steps for converting your scikit-learn models to the appropriate format, transpiling them to Orion Cairo, and deploying inference endpoints for prediction in AI Action.*

In this tutorial you will learn how to use the Giza tools though a XGBoost model.

## Before Starting
Before we start, ensure that you have installed the Giza stack, created a user, and logged-in. 

In [ ]:
! pipx install giza-cli # Install the Giza-CLI
! pip install giza-actions # Install the AI Actions SDK

! giza users create # Create a user
! giza users login # Login to your account
! giza users create-api-key # Create an API key. We recommend you do this so you don't have to reconnect.

## Create and Train an XGBoost Model
We'll start by creating a simple XGBoost model using Scikit-Learn and train it on diabetes dataset.

In [1]:
import xgboost as xgb
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split

data = load_diabetes()
X, y = data.data, data.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Increase the number of trees and maximum depth
n_estimators = 2  # Increase the number of trees
max_depth = 6  # Increase the maximum depth of each tree

xgb_reg = xgb.XGBRegressor(n_estimators=n_estimators, max_depth=max_depth)
xgb_reg.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=6, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=2, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

### Save model
Save the model in Json format

In [2]:
xgb_reg.save_model("xgb_diabetes.json")

## Transpile your model to Orion Cairo

We will use Giza-CLI to transpile our saved model to Orion Cairo.

In [3]:
! giza transpile xgb_diabetes.json --output-path xgb_diabetes

[giza][2024-05-10 17:14:48.565] No model id provided, checking if model exists ✅
[giza][2024-05-10 17:14:48.567] Model name is: xgb_diabetes
[giza][2024-05-10 17:14:49.081] Model already exists, using existing model ✅ 
[giza][2024-05-10 17:14:49.083] Model found with id -> 588! ✅
[giza][2024-05-10 17:14:49.777] Version Created with id -> 2! ✅
[giza][2024-05-10 17:14:49.780] Sending model for transpilation ✅ 
[giza][2024-05-10 17:15:00.670] Transpilation is fully compatible. Version compiled and Sierra is saved at Giza ✅
⠙ Transpiling Model...
[giza][2024-05-10 17:15:01.337] Downloading model ✅
[giza][2024-05-10 17:15:01.339] model saved at: xgb_diabetes


## Deploy an inference endpoint

Now that our model is transpiled to Cairo we can deploy an endpoint to run verifiable inferences. We will use Giza CLI again to run and deploy an endpoint.
Ensure to replace `model-id` and `version-id` with your ids provided during transpilation.

In [4]:
! giza endpoints deploy --model-id 588 --version-id 2

▰▰▰▰▰▰▰ Creating endpoint!t!
[giza][2024-05-10 17:15:21.628] Endpoint is successful ✅
[giza][2024-05-10 17:15:21.635] Endpoint created with id -> 190 ✅
[giza][2024-05-10 17:15:21.636] Endpoint created with endpoint URL: https://endpoint-raphael-doukhan-588-2-72c9b3b8-7i3yxzspbq-ew.a.run.app 🎉


## Run a verifiable inference in AI Actions

To streamline a verifiable inference, you might consider using the endpoint URL obtained after transpilation. However, this approach requires manual serialization of the input for the Cairo program and handling the deserialization process. To make this process more user-friendly and keep you within a Python environment, we've introduced AI Actions—a Python SDK designed to facilitate the creation of ML workflows and execution of verifiable predictions. When you initiate a prediction, our system automatically retrieves the endpoint URL you deployed earlier, converts your input into Cairo-compatible format, executes the prediction, and then converts the output back into a numpy object. More info about [AI Actions here.](https://actions.gizatech.xyz/)

First ensure you have an AI Actions workspace created. This step grants access to a user-friendly UI dashboard, enabling you to monitor and manage workflows with ease.

In [5]:
! giza workspaces get

# 🚨 If you haven't set up a workspace yet, you can establish one by executing the command below:
# `! giza workspaces create`

[giza][2024-05-10 17:15:39.231] Retrieving workspace information ✅ 
[giza][2024-05-10 17:15:39.534] ✅ Workspace URL: https://actions-server-raphael-doukhan-dblzzhtf5q-ew.a.run.app ✅
{
  "url": "https://actions-server-raphael-doukhan-dblzzhtf5q-ew.a.run.app",
  "status": "COMPLETED"
}


Now let's run a verifiable inference with AI Actions. To design your workflow in AI Actions, you will need to define your task with the `@task` decorator and then action your tasks with the `@action` decorator. You can track the progress of your workflow via the workspace URL previously provided.

In [7]:
from giza_actions.model import GizaModel
from giza_actions.action import action
from giza_actions.task import task

MODEL_ID = 588  # Update with your model ID
VERSION_ID = 2  # Update with your version ID


@task
def prediction(input, model_id, version_id):
    model = GizaModel(id=model_id, version=version_id)

    (result, proof_id) = model.predict(
        input_feed={'input': input}, verifiable=True, model_category="XGB"
    )

    return result, proof_id


@action
def execution():
    # The input data type should match the model's expected input
    input = X_test[1 , :]   

    (result, proof_id) = prediction(input, MODEL_ID, VERSION_ID)

    print(
        f"Predicted value for input {input.flatten()[0]} is {result}")

    return result, proof_id


execution()

/Users/raphaeldoukhan/Library/Caches/pypoetry/virtualenvs/xgb-diabetes-QT-DGK2y-py3.11/lib/python3.11/site-packages/prefect/tasks.py:332: UserWarning: A task named 'prediction' and defined at '/Users/raphaeldoukhan/Library/Caches/pypoetry/virtualenvs/xgb-diabetes-QT-DGK2y-py3.11/lib/python3.11/site-packages/giza_actions/task.py:9' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(
/Users/raphaeldoukhan/Library/Caches/pypoetry/virtualenvs/xgb-diabetes-QT-DGK2y-py3.11/lib/python3.11/site-packages/prefect/flows.py:336: UserWarning: A flow named 'execution' and defined at '/Users/raphaeldoukhan/Library/Caches/pypoetry/virtualenvs/xgb-diabetes-QT-DGK2y-py3.11/lib/python3.11/site-packages/giza_actions/action.py:20' conflicts with another flow. Consider specifying a unique `name` parameter in the flow definition:

 `@flow(name='my_unique_name', ...)`
  warnings.warn(


17:18:09.118 | INFO    | Created flow run 'versatile-harrier' for flow 'execution'

17:18:09.122 | INFO    | Action run 'versatile-harrier' - View at https://actions-server-raphael-doukhan-dblzzhtf5q-ew.a.run.app/flow-runs/flow-run/8b4c74cd-ca77-4433-a9c5-215ef922a7b8

17:18:09.455 | INFO    | Action run 'versatile-harrier' - Created task run 'prediction-0' for task 'prediction'

17:18:09.458 | INFO    | Action run 'versatile-harrier' - Executing 'prediction-0' immediately...

🚀 Starting deserialization process...
✅ Deserialization completed! 🎉


17:18:12.784 | INFO    | Task run 'prediction-0' - Finished in state Completed()

Predicted value for input 0.09256398319871433 is 175.58781


17:18:12.920 | INFO    | Action run 'versatile-harrier' - Finished in state Completed()

(175.58781, '546f8817fa454db78982463868440e8c')

## Download the proof

Initiating a verifiable inference sets off a proving job on our server, sparing you the complexities of installing and configuring the prover yourself. Upon completion, you can download your proof.

First, let's check the status of the proving job to ensure that it has been completed. 

🚨 Remember to substitute `endpoint-id` and `proof-id` with the specific IDs assigned to you throughout this tutorial.

In [16]:
! giza endpoints get-proof --endpoint-id 190 --proof-id 546f8817fa454db78982463868440e8c

[giza][2024-05-10 17:29:34.462] Getting proof from endpoint 190 ✅ 
{
  "id": 851,
  "job_id": 999,
  "metrics": {
    "proving_time": 14.691068
  },
  "created_date": "2024-05-10T14:20:08.682194"
}


Once the proof is ready, you can download it.

In [17]:
! giza endpoints download-proof --endpoint-id 190 --proof-id 546f8817fa454db78982463868440e8c --output-path zk_xgb.proof

[giza][2024-05-10 17:31:06.509] Getting proof from endpoint 190 ✅ 
[giza][2024-05-10 17:31:08.061] Proof downloaded to zk_xgb.proof ✅ 


## Verify the proof

Finally you can verify the proof.

In [3]:
! giza verify --proof zk_xgb.proof --size M

[giza][2024-05-10 17:40:36.948] Verification job created with name 'verify-cairo-20240510-93a8b208' and id -> 477 ✅
[giza][2024-05-10 17:42:18.151] Verification job is successful ✅m, elapsed 0.0s0s
